In [1]:
import os
import sys
this = os.getcwd()
path = this[:this.rfind("/")]
if not path in sys.path: sys.path.append(path)
xgboost_path = path + "/xgboost_regressor"
promedios_path = path + "/promedio_zona"

import pandas as pd
import matplotlib
from datos import FEATURES_DISPONIBLES
from modelo import Modelo

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from xgboost_regressor.xgboost_predictor import XGBoostRegressor
from promedio_zona.promedio_zona import PromedioZona

In [11]:
class EnsamblePromedioXGBoost(Modelo):
    """
        En principio, voy a poner un promedio de ambas predicciones.
    """
    
    @Modelo.cronometrar()
    def __init__(self):
        self.modelo_promedios = PromedioZona()
        self.modelo_xgboost = XGBoostRegressor()
        super().__init__()

    @Modelo.cronometrar()
    def cargar_datos(self):
        self.modelo_promedios.cargar_datos()
        self.modelo_xgboost.cargar_datos()
        self.cargado = True
    
    @Modelo.cronometrar()
    def entrenar(self):
        self.modelo_promedios.entrenar()
        self.modelo_xgboost.entrenar()
        super().entrenar()
    
    @Modelo.cronometrar()
    def validar(self):
        """
        """
        predicciones = self.predecir('test')
        score = self.puntuar(predicciones[self.feature], predicciones["target"])
        self.resultado_validacion = score
        self.validado = True
        return score

    @Modelo.cronometrar()
    def predecir(self, cual):
        """
            cual: {'test', 'submit'}
        """
        sets_disponibles  = {
            "test": {
                "promedios": self.modelo_promedios.test_data,
                "xgboost": self.modelo_xgboost.test_data
            },
            "submit": {
                "promedios": self.modelo_promedios.submit_data,
                "xgboost": self.modelo_xgboost.submit_data
            }
        }
        if cual not in sets_disponibles: raise Exception('No puedo predecir eso')
        
        columnas = [self.feature, 'target']
        prediccion_promedios = self.modelo_promedios.predecir(sets_disponibles.get(cual).get('promedios'))[columnas]
        prediccion_xgboost = self.modelo_xgboost.predecir(sets_disponibles.get(cual).get('xgboost'))[columnas]
        predicciones = prediccion_promedios.join(prediccion_xgboost, lsuffix='_promedio', rsuffix='_xgboost')
        predicciones['target'] = predicciones[['target_promedio', 'target_xgboost']].mean(axis='columns')
        predicciones[self.feature] = predicciones[[self.feature+'_promedio', self.feature+'_xgboost']].mean(axis='columns')
        return predicciones    


In [12]:
ensamble = EnsamblePromedioXGBoost()

instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
__init__ demoro 0.0 segundos


In [13]:
ensamble.cargar_datos()

cargar demoro 1.37 segundos
cargar demoro 1.73 segundos
cargar_datos demoro 7.49 segundos


In [14]:
ensamble.entrenar()

entrenar demoro 0.0 segundos
entrenar demoro 0.15 segundos
[12:05:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
entrenar demoro 0.0 segundos
entrenar demoro 279.21 segundos
entrenar demoro 0.0 segundos
entrenar demoro 279.35 segundos


In [15]:
ensamble.validar()

predecir demoro 6.8 segundos
predecir demoro 3.02 segundos
predecir demoro 9.83 segundos
puntuar demoro 0.0 segundos
validar demoro 9.83 segundos


702184.908644387

In [ ]:
comentario = "test ensamble promedios + xgboost"
ensamble.presentar(predicciones, comentario)